In [ ]:
## Module import
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from scipy import stats
import seaborn as sns
import cartopy.feature as cft
import matplotlib.path as mpath
import cmocean as cm
import string
import math
from datetime import datetime, timedelta
import proplot as pplt
import xesmf as xe
import netCDF4 as nc

## 1) Standard deviation

In [ ]:
# open IPSL file
netCDF_T55_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_T_for_std_19552014.nc'
TEMP_CMIP55 = xr.open_mfdataset(netCDF_T55_file)
TEMP_CMIP55

netCDF_S55_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_S_for_std_19552014.nc'
SAL_CMIP55 = xr.open_mfdataset(netCDF_S55_file)
SAL_CMIP55


# open MPI file
netCDF_MPI_T55_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_T_for_std_19552014.nc'
TEMP_MPI_CMIP55 = xr.open_mfdataset(netCDF_MPI_T55_file)
TEMP_MPI_CMIP55

netCDF_MPI_S55_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_S_for_std_19552014.nc'
SAL_MPI_CMIP55 = xr.open_mfdataset(netCDF_MPI_S55_file)
SAL_MPI_CMIP55


# open UKESM file
netCDF_UKESM_T55_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_T_for_std_19552014.nc'
TEMP_UKESM_CMIP55 = xr.open_mfdataset(netCDF_UKESM_T55_file)
TEMP_UKESM_CMIP55

netCDF_UKESM_S55_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_S_for_std_19552014.nc'
SAL_UKESM_CMIP55 = xr.open_mfdataset(netCDF_UKESM_S55_file)
SAL_UKESM_CMIP55

# open topo
netCDF_topom_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/deptho_Ofx_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn.nc'
TOPOM = xr.open_mfdataset(netCDF_topom_file)

netCDF_topou_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/deptho_Ofx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc'
TOPOU = xr.open_mfdataset(netCDF_topou_file)

# correction topo pour contour
delta_lon = np.abs(np.diff(TOPOM['longitude']))
for i, start in enumerate(np.argmax(delta_lon > 180, axis=1)):
    TOPOM['longitude'][i, start+1:] += 360
    
delta_lon1 = np.abs(np.diff(TOPOU['longitude']))
for i, start in enumerate(np.argmax(delta_lon1 > 180, axis=1)):
    TOPOU['longitude'][i, start+1:] += 360
    
netCDF_area = '/Users/jcaillet/Documents/Elmer/DATA/BATHY/BedMachineAntarctica_v02_m.nc'
Area = xr.open_mfdataset(netCDF_area)

## 2) T and S Profils

### 2.1) Observations

In [ ]:
############################################ Open files ##########################################################

# Temperature 1955-2017
netCDF_fileT2 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/T/woa18_1955-2017_t00_01.nc'
OBST55 = xr.open_mfdataset(netCDF_fileT2, decode_cf=False)
OBST55 = OBST55.squeeze()

# Salinity 1955-2017
netCDF_fileS2 = '/Users/jcaillet/Documents/Elmer/CMIP6/OBSERVATIONS/S/woa18_1955-2017_s00_01.nc'
OBSS55 = xr.open_mfdataset(netCDF_fileS2, decode_cf=False)
OBSS55 = OBSS55.squeeze()

# mask file of Amundsen Sea
netCDF_fileT = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/00_mask_continental_shelf.nc'
Mask = xr.open_mfdataset(netCDF_fileT)

# grid file of Amundsen Sea
netCDF_grid_file = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/mesh_mask_AMUXL025_BedMachineAntarctica-2020-07-15_v02_ICB380.nc'
Grid = xr.open_mfdataset(netCDF_grid_file).squeeze()

##################################  Interpolation of Amundsen sea mask on grid 1°x1° ##############################

# add lon, lat to mask file
Grid['Mask'] = Grid['mbathy'] - Grid['mbathy'] + Mask.isfmask.values
Grid = Grid.rename({"nav_lon": "lon", "nav_lat": "lat"})

# interpolation on grid 1°x1° (same as Observation)
## target Grid
ds_out = xe.util.grid_global(1, 1)
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, ds_out, "bilinear")
## regridding
dr_out = regridder(dr)
## rename
dr_out = dr_out.rename({"lon": "nav_lon", "lat": "nav_lat"})
dr_out = dr_out.rename({"x": "lon", "y": "lat"})

# add to data
OBST55['mask'] = dr_out
OBSS55['mask'] = dr_out

###################################### Mean profil on the continental shelf #######################################

# non-value : np.nan instead of 1e36
OBST55['temp']= xr.where(OBST55.t_an>1000,np.nan,OBST55.t_an.values)
OBST55['depth'] = - OBST55['depth']

OBSS55['sal'] = xr.where(OBSS55.s_an>1000,np.nan,OBSS55.s_an.values)
OBSS55['depth'] = - OBSS55['depth']

# mean T profil on continental shelf
OBST55_Amundsen = OBST55.temp.where(OBST55.mask==1).mean(['lon','lat']) ## approximation cells have the same size

# mean S profil on continental shelf
OBSS55_Amundsen = OBSS55.sal.where(OBSS55.mask==1).mean(['lon','lat']) ## approximation cells have the same size

### 2.2) CMIP6 outputs

#### 2.2.1) IPSL-CM6A-LR model

In [ ]:
# Temperature & salinity 1955-2014
netCDF = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/profil_T55_IPSL.nc'
CMIPT55I = xr.open_mfdataset(netCDF,concat_dim='members',combine='nested')
CMIPT55I = CMIPT55I.sortby('members')

netCDFS = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/profil_S55_IPSL.nc'
CMIPS55I = xr.open_mfdataset(netCDFS,concat_dim='members',combine='nested')
CMIPS55I = CMIPS55I.sortby('members')

# Area of cells
netCDF_area = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/areacello_Ofx_IPSL-CM6A-LR_historical_r1i1p1f1_gn.nc'
Area = xr.open_mfdataset(netCDF_area).sel(y=slice(0,100), x=slice(100,250))

# mask file of Amundsen Sea
netCDF_fileT = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/00_mask_continental_shelf.nc'
Mask = xr.open_mfdataset(netCDF_fileT)

# grid file of Amundsen Sea
netCDF_grid_file = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/mesh_mask_AMUXL025_BedMachineAntarctica-2020-07-15_v02_ICB380.nc'
Grid = xr.open_mfdataset(netCDF_grid_file).squeeze()

# add lon, lat to mask file
Grid['Mask'] = Grid['mbathy'] - Grid['mbathy'] + Mask.isfmask.values
Grid = Grid.rename({"nav_lon": "lon", "nav_lat": "lat"})

# Interpolation of Amundsen Mask on the CMIP6 Grid
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, CMIPT55I, "bilinear")
## regridding
dr_out = regridder(dr)


# add mask to data
CMIPT55I['mask'] = CMIPT55I['nav_lon'] - CMIPT55I['nav_lon'] + dr_out.values
CMIPT55I['area'] = Area['areacello'].broadcast_like(CMIPT55I['thetao'])
CMIPT55I['area'] = CMIPT55I['area'] * CMIPT55I['thetao'] / CMIPT55I['thetao']

# vertical profil
CMIPT55I_mean = (CMIPT55I['thetao'] * CMIPT55I['area']).where(CMIPT55I['mask']==1).sum(['x','y'])/ CMIPT55I['area'].where(CMIPT55I['mask']==1).sum(['x','y'])
CMIPT55I_mean['olevel']= - CMIPT55I_mean['olevel']

CMIPS55I_mean = (CMIPS55I['so'] * CMIPT55I['area']).where(CMIPT55I['mask']==1).sum(['x','y'])/ CMIPT55I['area'].where(CMIPT55I['mask']==1).sum(['x','y'])
CMIPS55I_mean['olevel']= - CMIPS55I_mean['olevel']

# mean members
CMIPT55I_mean_tot = CMIPT55I_mean.mean('members')
CMIPS55I_mean_tot = CMIPS55I_mean.mean('members')

#### 2.2.2) MPI-ESM1.2-HR model

In [ ]:
# Temperature & salinity 1955-2014
netCDF = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/profil_T55_MPI.nc'
CMIPT55M = xr.open_mfdataset(netCDF,concat_dim='members',combine='nested')
CMIPT55M = CMIPT55M.sortby('members')

netCDFS = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/profil_S55_MPI.nc'
CMIPS55M = xr.open_mfdataset(netCDFS,concat_dim='members',combine='nested')
CMIPS55M = CMIPS55M.sortby('members')

# Area of cells
netCDF_area = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/areacello_Ofx_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn.nc'
Area = xr.open_mfdataset(netCDF_area).sel(j=slice(380,400), i=slice(300,410))

# mask file of Amundsen Sea
netCDF_fileT = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/00_mask_continental_shelf.nc'
Mask = xr.open_mfdataset(netCDF_fileT)

# grid file of Amundsen Sea
netCDF_grid_file = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/mesh_mask_AMUXL025_BedMachineAntarctica-2020-07-15_v02_ICB380.nc'
Grid = xr.open_mfdataset(netCDF_grid_file).squeeze()

# add lon, lat to mask file
Grid['Mask'] = Grid['mbathy'] - Grid['mbathy'] + Mask.isfmask.values
Grid = Grid.rename({"nav_lon": "lon", "nav_lat": "lat"})

# Interpolation of Amundsen Mask on the CMIP6 Grid
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, CMIPT55M, "bilinear")
## regridding
dr_out = regridder(dr)

# add mask to data
CMIPT55M['mask'] = CMIPT55M['longitude'] - CMIPT55M['longitude'] + dr_out.values
CMIPT55M['area'] = Area['areacello'].broadcast_like(CMIPT55M['thetao'])
CMIPT55M['area'] = CMIPT55M['area'] * CMIPT55M['thetao'] / CMIPT55M['thetao']

# vertical profil
CMIPT55M_mean = (CMIPT55M['thetao'] * CMIPT55M['area']).where(CMIPT55M['mask']==1).sum(['i','j'])/ CMIPT55M['area'].where(CMIPT55M['mask']==1).sum(['i','j'])
CMIPT55M_mean['lev']= - CMIPT55M_mean['lev']

CMIPS55M_mean = (CMIPS55M['so'] * CMIPT55M['area']).where(CMIPT55M['mask']==1).sum(['i','j'])/ CMIPT55M['area'].where(CMIPT55M['mask']==1).sum(['i','j'])
CMIPS55M_mean['lev']= - CMIPS55M_mean['lev']

# mean members
CMIPT55M_mean_tot = CMIPT55M_mean.mean('members')
CMIPS55M_mean_tot = CMIPS55M_mean.mean('members')

#### 2.2.3) UKESM1-0-LL model

In [ ]:
# Temperature & salinity 1955-2014
netCDF = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/profil_T55_UKESM.nc'
CMIPT55U = xr.open_mfdataset(netCDF,concat_dim='members',combine='nested')
CMIPT55U = CMIPT55U.sortby('members')

netCDFS = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/profil_S55_UKESM.nc'
CMIPS55U = xr.open_mfdataset(netCDFS,concat_dim='members',combine='nested')
CMIPS55U = CMIPS55U.sortby('members')

# Area of cells
netCDF_area = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/areacello_Ofx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc'
Area = xr.open_mfdataset(netCDF_area).sel(j=slice(30,80), i=slice(130,200))

# mask file of Amundsen Sea
netCDF_fileT = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/00_mask_continental_shelf.nc'
Mask = xr.open_mfdataset(netCDF_fileT)

# grid file of Amundsen Sea
netCDF_grid_file = '/Users/jcaillet/Documents/nemo/AMUXL025/OTHERS/mesh_mask_AMUXL025_BedMachineAntarctica-2020-07-15_v02_ICB380.nc'
Grid = xr.open_mfdataset(netCDF_grid_file).squeeze()

# add lon, lat to mask file
Grid['Mask'] = Grid['mbathy'] - Grid['mbathy'] + Mask.isfmask.values
Grid = Grid.rename({"nav_lon": "lon", "nav_lat": "lat"})

# Interpolation of Amundsen Mask on the CMIP6 Grid
## intial Grid
dr = Grid.Mask
## method of interpolation
regridder = xe.Regridder(Grid, CMIPT55U, "bilinear")
## regridding
dr_out = regridder(dr)

# add mask to data
CMIPT55U['mask'] = CMIPT55U['longitude'] - CMIPT55U['longitude'] + dr_out.values
CMIPT55U['area'] = Area['areacello'].broadcast_like(CMIPT55U['thetao'])
CMIPT55U['area'] = CMIPT55U['area'] * CMIPT55U['thetao'] / CMIPT55U['thetao']

# vertical profil
CMIPT55U_mean = (CMIPT55U['thetao'] * CMIPT55U['area']).where(CMIPT55U['mask']==1).sum(['i','j'])/ CMIPT55U['area'].where(CMIPT55U['mask']==1).sum(['i','j'])
CMIPT55U_mean['lev']= - CMIPT55U_mean['lev']

CMIPS55U_mean = (CMIPS55U['so'] * CMIPT55U['area']).where(CMIPT55U['mask']==1).sum(['i','j'])/ CMIPT55U['area'].where(CMIPT55U['mask']==1).sum(['i','j'])
CMIPS55U_mean['lev']= - CMIPS55U_mean['lev']

# mean members
CMIPT55U_mean_tot = CMIPT55U_mean.mean('members')
CMIPS55U_mean_tot = CMIPS55U_mean.mean('members')

### 3) Figure

In [ ]:
sns.set_context('paper')

I = np.arange(0,33,1)
M = np.arange(0,10,1)
U = np.arange(0,16,1)

land_50m = cft.NaturalEarthFeature('physical', 'land', '50m', edgecolor='black', facecolor='papayawhip', linewidth=0.5)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

fig, axs = plt.subplots(nrows = 4, ncols = 3, subplot_kw={'projection': ccrs.SouthPolarStereo()}, figsize = (7, 9), constrained_layout=True)
axs = axs.flatten()

for n, ax in enumerate(axs):
    ax.text(0.025, 0.9, string.ascii_lowercase[n]+')', transform=ax.transAxes, color='white', fontweight="bold")

for ax in np.array([axs[0],axs[1], axs[2],axs[3],axs[4],axs[5]]):
    ax.set_extent([-180, 180, -90, -63], crs=ccrs.PlateCarree())
    ax.add_feature(land_50m, color=[0.8, 0.8, 0.8])
    ax.coastlines(resolution='50m')
    gl0 = ax.gridlines(draw_labels=False, linewidth=0.75, color='gray', alpha=0.7, linestyle='--', dms=True, x_inline=False, y_inline=False)
    gl0.top_labels = False
    gl0.right_labels = False
    gl0.xlocator = mticker.FixedLocator([-180, -135, -90, -45, 0, 45, 90, 135, 180])
    gl0.ylocator = mticker.FixedLocator([-90, -80, -70, -60])
    gl0.xformatter = LongitudeFormatter()
    gl0.yformatter = LatitudeFormatter()
    gl0.xlabel_style = {'size': 12, 'color': 'gray'}
    gl0.ylabel_style = {'size': 12, 'color': 'gray'}


# Salinity 1955-2014
p4 = axs[0].pcolormesh(SAL_MPI_CMIP55.longitude, SAL_MPI_CMIP55.latitude, SAL_MPI_CMIP55['Sal_std_200700'],  cmap=cm.cm.haline,  vmin=0, vmax=0.06, transform=ccrs.PlateCarree(), shading='auto')
p44 = axs[0].contour(TOPOM.longitude, TOPOM.latitude, TOPOM['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

p3 = axs[1].pcolormesh(SAL_CMIP55.nav_lon, SAL_CMIP55.nav_lat, SAL_CMIP55['Sal_std_200700'],  cmap=cm.cm.haline, vmin=0, vmax=0.06, transform=ccrs.PlateCarree(), shading='auto')
p33 = axs[1].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

p5 = axs[2].pcolormesh(SAL_UKESM_CMIP55.longitude, SAL_UKESM_CMIP55.latitude, SAL_UKESM_CMIP55['Sal_std_200700'],  cmap=cm.cm.haline,  vmin=0, vmax=0.06, transform=ccrs.PlateCarree(), shading='auto')
p55 = axs[2].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='salmon', linewidths = 1, transform=ccrs.PlateCarree())

# Temperature 1955-2014
p1 = axs[3].pcolormesh(TEMP_MPI_CMIP55.longitude, TEMP_MPI_CMIP55.latitude, TEMP_MPI_CMIP55['Temp_std_200700'], cmap=cm.cm.thermal, vmin=0, vmax=0.4, transform=ccrs.PlateCarree(), shading='auto')
p11 = axs[3].contour(TOPOM.longitude, TOPOM.latitude, TOPOM['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

p0 = axs[4].pcolormesh(TEMP_CMIP55.nav_lon, TEMP_CMIP55.nav_lat, TEMP_CMIP55['Temp_std_200700'], cmap=cm.cm.thermal, vmin=0, vmax=0.4, transform=ccrs.PlateCarree(), shading='auto')
p00 = axs[4].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

p2 = axs[5].pcolormesh(TEMP_UKESM_CMIP55.longitude, TEMP_UKESM_CMIP55.latitude, TEMP_UKESM_CMIP55['Temp_std_200700'], cmap=cm.cm.thermal, vmin=0, vmax=0.4, transform=ccrs.PlateCarree(), shading='auto')
p22 = axs[5].contour(TOPOU.longitude, TOPOU.latitude, TOPOU['deptho'], levels=[1500], colors='lightskyblue', linewidths = 1, transform=ccrs.PlateCarree())

ax_cb = plt.axes([0.087, 0.55, 0.01, 0.15])
cb = plt.colorbar(p2, cax=ax_cb, orientation='vertical')
cb.ax.set_ylabel('Standard deviation (°C)');

ax_cb1 = plt.axes([0.085, 0.79, 0.01, 0.15])
cb1 = plt.colorbar(p5, cax=ax_cb1, orientation='vertical')
cb1.ax.set_ylabel('Standard deviation (g kg$^{-1}$)');
  
plt.figtext(0.01, 0.865, 'Across member standard', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.03, 0.865, 'deviation of the 1955-2014', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.05, 0.865, 'mean practical salinity', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.01, 0.625, 'Across member standard', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.03, 0.625, 'deviation of the 1955-2014', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.05, 0.625, 'mean potential temperature', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.28, 0.97, 'MPI', fontweight="bold")
plt.figtext(0.56, 0.97, 'IPSL', fontweight="bold")
plt.figtext(0.82, 0.97, 'UKESM', fontweight="bold")
plt.figtext(0.01, 0.385, 'Salinity over the', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.03, 0.385, 'Amundsen Sea', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.05, 0.385, 'continental shelf', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.01, 0.145, 'Temperature over the', horizontalalignment='left', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.03, 0.145, 'Amundsen Sea', horizontalalignment='left', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.05, 0.145, 'continental shelf', horizontalalignment='left', verticalalignment='center', rotation=90, fontweight="bold")


## Temperature 1955-2014 - MPI
ax6 = fig.add_subplot(4,3,10)
#WOA
ax6.plot(OBST55_Amundsen, OBST55_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1955-2017')
#CMIP6
for k in M:
    if k == 0:
        ax6.plot(CMIPT55M_mean[k,:], CMIPT55M_mean.lev, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax6.plot(CMIPT55M_mean[k,:], CMIPT55M_mean.lev, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax6.plot(CMIPT55M_mean_tot, CMIPT55M_mean_tot.lev, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax6.text(0.9, 0.9, 'j)', transform=ax6.transAxes, fontweight="bold")
ax6.set_ylabel('Depth (m)')
ax6.set_xlim(-2,2)
ax6.set_ylim(-2000,0)
plt.grid(True)

## Temperature 1955-2014 - IPSL
ax7 = fig.add_subplot(4,3,11)
#WOA
ax7.plot(OBST55_Amundsen, OBST55_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1995-2017')
#CMIP6
for k in I:
    if k == 0:
        ax7.plot(CMIPT55I_mean[k,:], CMIPT55I_mean.olevel, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax7.plot(CMIPT55I_mean[k,:], CMIPT55I_mean.olevel, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax7.plot(CMIPT55I_mean_tot, CMIPT55I_mean_tot.olevel, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax7.text(0.9, 0.9, 'k)', transform=ax7.transAxes, fontweight="bold")
ax7.set_xlabel('Potential Temperature (°C)')
plt.legend(loc = 'lower left', prop={"size":8})
ax7.set_xlim(-2,2)
ax7.set_ylim(-2000,0)
plt.grid(True)
ax7.axes.yaxis.set_ticklabels([])


## Temperature 1955-2014 - UKESM
ax8 = fig.add_subplot(4,3,12)
#WOA
ax8.plot(OBST55_Amundsen, OBST55_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1955-2017')
#CMIP6
for k in U:
    if k == 0:
        ax8.plot(CMIPT55U_mean[k,:], CMIPT55U_mean.lev, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax8.plot(CMIPT55U_mean[k,:], CMIPT55U_mean.lev, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax8.plot(CMIPT55U_mean_tot, CMIPT55U_mean_tot.lev, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax8.text(0.9, 0.9, 'l)', transform=ax8.transAxes, fontweight="bold")
ax8.set_xlim(-2,2)
ax8.set_ylim(-2000,0)
plt.grid(True)
ax8.axes.yaxis.set_ticklabels([])


## Salinity 1955-2014 - MPI
ax9 = fig.add_subplot(4,3,7)
#WOA
ax9.plot(OBSS55_Amundsen, OBSS55_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1955-2017')
#CMIP6
for k in M:
    if k == 0:
        ax9.plot(CMIPS55M_mean[k,:], CMIPS55M_mean.lev, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax9.plot(CMIPS55M_mean[k,:], CMIPS55M_mean.lev, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax9.plot(CMIPS55M_mean_tot, CMIPS55M_mean_tot.lev, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax9.text(0.9, 0.9, 'g)', transform=ax9.transAxes, fontweight="bold")
ax9.set_ylabel('Depth (m)')
ax9.set_xlim(33.5,34.8)
ax9.set_ylim(-2000,0)
plt.grid(True)

## Salinity 1955-2014 - IPSL
ax10 = fig.add_subplot(4,3,8)
#WOA
ax10.plot(OBSS55_Amundsen, OBSS55_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1955-2017')
#CMIP6
for k in I:
    if k == 0:
        ax10.plot(CMIPS55I_mean[k,:], CMIPS55I_mean.olevel, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax10.plot(CMIPS55I_mean[k,:], CMIPS55I_mean.olevel, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax10.plot(CMIPS55I_mean_tot, CMIPS55I_mean_tot.olevel, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax10.text(0.9, 0.9, 'h)', transform=ax10.transAxes, fontweight="bold")
ax10.set_xlabel('Practical Salinity (g kg$^{-1}$)')
plt.legend(loc = 'lower left', prop={"size":8})
ax10.set_xlim(33.5,34.8)
ax10.set_ylim(-2000,0)
plt.grid(True)
ax10.axes.yaxis.set_ticklabels([])

## Salinity 1955-2014 - UKESM
ax11 = fig.add_subplot(4,3,9)
#WOA
ax11.plot(OBSS55_Amundsen, OBSS55_Amundsen.depth, color = 'grey', linewidth = 1.5, label= 'WOA18:1995-2017')
#CMIP6
for k in U:
    if k == 0:
        ax11.plot(CMIPS55U_mean[k,:], CMIPS55U_mean.lev, color = 'dodgerblue', linewidth = 0.8, label= 'CMIP6 members')
    else :
        ax11.plot(CMIPS55U_mean[k,:], CMIPS55U_mean.lev, color = 'dodgerblue', linewidth = 0.8)
#mean CMIP6
ax11.plot(CMIPS55U_mean_tot, CMIPS55U_mean_tot.lev, color = 'crimson', linewidth = 1.5, label= 'CMIP6 mean')

ax11.text(0.9, 0.9, 'i)', transform=ax11.transAxes, fontweight="bold")
ax11.set_xlim(33.5,34.8)
ax11.set_ylim(-2000,0)
plt.grid(True)
ax11.axes.yaxis.set_ticklabels([])

plt.tight_layout(pad = 1.0,rect=[0.07, 0, 1, 0.97])

plt.savefig('/Users/jcaillet/Documents/Elmer/CMIP6/FIGURES/Figure2_std19552014_modif.png',dpi=300)